In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from utils.helpers import *
import keras
from keras import layers
from keras import models
from keras import optimizers
import numpy as np
from matplotlib.pyplot import imshow
import h5py
%matplotlib inline
keras.__version__

Using TensorFlow backend.


'2.1.3'

In [2]:
IMAGE_ID = 3
PADDING = 47
image, mask = get_image_mask_from_id(IMAGE_ID)
image = image / 255.

In [3]:
test = apply_padding(image, padding=PADDING)
H, W, C = test.shape
type(H)
type(W)
type(C)

int

In [ ]:
old_model = models.load_model("train_state_2.h5")

In [ ]:
Ws = []
bs = []
for layer in old_model.layers:
    if len(layer.get_weights()) > 0:
        W, b = layer.get_weights()
        Ws.append(W)
        bs.append(b)
        
Ws[4] = Ws[4].reshape(4,4,256,200)
Ws[5] = Ws[5].reshape(1,1,200,3)

# for w in Ws:
#     print(w.shape)
# print(Ws)

In [9]:
img_input = layers.Input(shape=(95, 95, 3))
x = layers.Conv2D(32, (1, 1), activation='relu', name="CONV1")(img_input)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2), name="MP1")(x)

x = layers.Conv2D(64, (3, 3), activation='relu', name="CONV2")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), name="MP2")(x)

x = layers.Conv2D(128, (3, 3), activation='relu', name="CONV3")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2), name="MP3")(x)

x = layers.Conv2D(256, (3, 3), activation='relu', name="CONV4")(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(200, activation='relu', name='DENSE')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation='softmax', name='SOFTMAX')(x)
model = models.Model(input = img_input, output = x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 95, 95, 3)         0         
_________________________________________________________________
CONV1 (Conv2D)               (None, 95, 95, 32)        128       
_________________________________________________________________
batch_normalization_17 (Batc (None, 95, 95, 32)        128       
_________________________________________________________________
MP1 (MaxPooling2D)           (None, 47, 47, 32)        0         
_________________________________________________________________
CONV2 (Conv2D)               (None, 45, 45, 64)        18496     
_________________________________________________________________
batch_normalization_18 (Batc (None, 45, 45, 64)        256       
_________________________________________________________________
MP2 (MaxPooling2D)           (None, 22, 22, 64)        0         
__________

/Users/csaravia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("SO...)`


In [5]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [16]:
img_input = layers.Input(shape=(H, W, 3))
x = layers.Conv2D(32, (1, 1), activation='relu', name="CONV1")(img_input)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2), name="MP1")(x)

x = layers.Conv2D(64, (3, 3), activation='relu', name="CONV2")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), name="MP2")(x)

x = layers.Conv2D(128, (3, 3), activation='relu', name="CONV3")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2,2), name="MP3")(x)

x = layers.Conv2D(256, (3, 3), activation='relu', name="CONV4")(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(200, (8, 8), activation='relu', name="CONV5")(x)
x = layers.Conv2D(3, (1, 1), activation='relu', name="CONV6")(x)

model = models.Model(input = img_input, output = x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 350, 350, 3)       0         
_________________________________________________________________
CONV1 (Conv2D)               (None, 350, 350, 32)      128       
_________________________________________________________________
batch_normalization_45 (Batc (None, 350, 350, 32)      128       
_________________________________________________________________
MP1 (MaxPooling2D)           (None, 175, 175, 32)      0         
_________________________________________________________________
CONV2 (Conv2D)               (None, 173, 173, 64)      18496     
_________________________________________________________________
batch_normalization_46 (Batc (None, 173, 173, 64)      256       
_________________________________________________________________
MP2 (MaxPooling2D)           (None, 86, 86, 64)        0         
__________

/Users/csaravia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("CO...)`


In [ ]:
# Predict using model
tests = []
tests.append(test)
tests = np.array(tests)
predictions = model.predict(tests)

In [ ]:
prediction = predictions[0]

imshow(prediction)

In [ ]:
pred = np.argmax(prediction, axis = -1)
imshow(pred)

In [ ]:
prediction